# Tutorial 5: Current Compensation Design with GAPoT

## Diseño de Compensación de Corriente con GAPoT

This tutorial demonstrates how to use GAPoT for designing power factor correction:

1. **Current decomposition**: Active vs non-active components
2. **Geometric projection/rejection**: The math behind compensation
3. **Compensation strategies**: Full vs partial compensation
4. **Practical considerations**: Trade-offs and limitations

---

## Theoretical Background

### The Compensation Problem

Given a load drawing current **i** from voltage **u**, we want to:
1. Reduce reactive power consumption
2. Minimize harmonic distortion injection
3. Achieve unity power factor if possible

### GA Solution: Projection and Rejection

The current can be decomposed geometrically:

$$\mathbf{i} = \mathbf{i}_P + \mathbf{i}_N$$

where:
- **Active current**: $\mathbf{i}_P = \text{proj}_{\mathbf{u}}(\mathbf{i}) = \frac{\mathbf{i} \cdot \mathbf{u}}{\|\mathbf{u}\|^2} \mathbf{u}$
- **Non-active current**: $\mathbf{i}_N = \mathbf{i} - \mathbf{i}_P$

### Unity Power Factor Condition

For PF = 1, the supply current must be collinear with voltage:

$$\mathbf{i}_{supply} = \mathbf{i}_P = G_e \mathbf{u}$$

where $G_e = P / \|\mathbf{u}\|^2$ is the equivalent conductance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from gapot import GeometricPower
from gapot.compensation import CurrentDecomposition, project_onto, reject_from
from gapot.utils import generate_distorted_signal, rms

plt.style.use('seaborn-v0_8-whitegrid')
print("Modules loaded!")

## 1. Setup: Distorted Load Current

In [ ]:
# System parameters
f1 = 50.0
fs = 20000.0
cycles = 4

# Load with poor power factor and harmonics
U1 = 230.0
I1 = 20.0
phi1 = 0.7  # ~45° lagging, PF ≈ 0.76

harmonics = [
    (5, 0.03, 0.25, 0.5),   # 25% 5th harmonic current
    (7, 0.02, 0.15, 0.3),   # 15% 7th harmonic current
]

u, i, t = generate_distorted_signal(
    U1=U1, I1=I1, phi1=phi1,
    harmonics=harmonics,
    f1=f1, fs=fs, cycles=cycles
)

print(f"Load voltage RMS: {rms(u):.2f} V")
print(f"Load current RMS: {rms(i):.2f} A")

In [ ]:
# Analyze BEFORE compensation
gp_before = GeometricPower(u, i, f1=f1, fs=fs)

print("BEFORE COMPENSATION")
print("=" * 50)
print(f"Active Power P        = {gp_before.P:.2f} W")
print(f"Reactive Power ||M_Q||= {gp_before.M_Q_norm:.2f} var")
print(f"Distortion Power ||M_D||= {gp_before.M_D_norm:.2f} va")
print(f"Apparent Power S      = {gp_before.S:.2f} VA")
print(f"Power Factor          = {gp_before.PF:.4f}")
print("=" * 50)

## 2. Current Decomposition

In [ ]:
# Decompose current using GA vectors
decomp = CurrentDecomposition(gp_before.u_vec, gp_before.i_vec)

print(decomp.summary())

In [ ]:
# Visualize decomposition in vector space (2D projection for first harmonic)
fig, ax = plt.subplots(figsize=(10, 10))

# Scale factors for visualization
scale_u = 1.0
scale_i = 10.0  # Scale current for visibility

# Get first two components (fundamental cos and sin)
u_2d = gp_before.u_vec[:2] * scale_u
i_2d = gp_before.i_vec[:2] * scale_i
i_P_2d = decomp.i_active[:2] * scale_i
i_N_2d = decomp.i_reactive[:2] * scale_i

# Plot vectors
ax.arrow(0, 0, u_2d[0], u_2d[1], head_width=8, head_length=5, 
         fc='blue', ec='blue', linewidth=2, label='Voltage u')
ax.arrow(0, 0, i_2d[0], i_2d[1], head_width=8, head_length=5,
         fc='red', ec='red', linewidth=2, label='Current i (×10)')
ax.arrow(0, 0, i_P_2d[0], i_P_2d[1], head_width=8, head_length=5,
         fc='green', ec='green', linewidth=2, label='Active i_P (×10)')
ax.arrow(0, 0, i_N_2d[0], i_N_2d[1], head_width=8, head_length=5,
         fc='orange', ec='orange', linewidth=2, label='Non-active i_N (×10)')

# Show i_N starts from i_P
ax.arrow(i_P_2d[0], i_P_2d[1], i_N_2d[0], i_N_2d[1], head_width=5, head_length=3,
         fc='orange', ec='orange', linewidth=1.5, alpha=0.5, linestyle='--')

# Annotations
ax.annotate('u', xy=(u_2d[0]/2, u_2d[1]/2 + 15), fontsize=14, color='blue')
ax.annotate('i', xy=(i_2d[0]/2 + 10, i_2d[1]/2), fontsize=14, color='red')
ax.annotate('i_P', xy=(i_P_2d[0]/2, i_P_2d[1]/2 - 15), fontsize=14, color='green')
ax.annotate('i_N', xy=(i_N_2d[0]/2 - 20, i_N_2d[1]/2), fontsize=14, color='orange')

# Draw right angle marker
angle_size = 20
ax.plot([i_P_2d[0], i_P_2d[0] - angle_size * i_N_2d[1]/np.linalg.norm(i_N_2d)],
        [i_P_2d[1], i_P_2d[1] + angle_size * i_N_2d[0]/np.linalg.norm(i_N_2d)], 'k-', linewidth=1)

ax.set_xlim(-50, 350)
ax.set_ylim(-200, 100)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.set_xlabel('Cosine component')
ax.set_ylabel('Sine component')
ax.set_title('Current Decomposition: i = i_P + i_N\n(Fundamental frequency projection)')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: i_P is parallel to u, i_N is perpendicular.")
print("Compensation means injecting -i_N to cancel the non-active component.")

## 3. Design Compensation Current

In [ ]:
# Get compensation current (negative of non-active)
i_comp_vec = decomp.get_compensation_current(
    compensate_reactive=True,
    compensate_distortion=True
)

# Reconstruct time-domain compensation current
# Using inverse Fourier transform
from gapot.basis import FourierToGA

converter = FourierToGA(fs=fs, f1=f1)
converter.transform(u)  # Setup frequency components

# For demonstration, we'll calculate i_comp in time domain directly
# i_comp = -i_N where i_N = i - i_P

# Calculate equivalent conductance
G_e = gp_before.P / (rms(u)**2)
print(f"Equivalent conductance G_e = P/U²_rms = {G_e:.6f} S")

# Active current in time domain
i_active_time = G_e * u

# Non-active current
i_nonactive_time = i - i_active_time

# Compensation current (inject negative of non-active)
i_comp_time = -i_nonactive_time

# Supply current after compensation
i_supply = i + i_comp_time

print(f"\nCompensation current RMS: {rms(i_comp_time):.2f} A")
print(f"Supply current RMS after compensation: {rms(i_supply):.2f} A")

In [ ]:
# Plot time-domain currents
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

t_ms = t * 1000

# Original load current
axes[0].plot(t_ms, i, 'r-', linewidth=1.5, label='Load current i')
axes[0].plot(t_ms, i_active_time, 'g--', linewidth=1, label='Active component i_P')
axes[0].set_ylabel('Current (A)')
axes[0].set_title('Original Load Current and Active Component')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Compensation current
axes[1].plot(t_ms, i_nonactive_time, 'orange', linewidth=1.5, label='Non-active i_N')
axes[1].plot(t_ms, i_comp_time, 'purple', linewidth=1.5, linestyle='--', label='Compensation i_comp = -i_N')
axes[1].set_ylabel('Current (A)')
axes[1].set_title('Non-Active Current and Required Compensation')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Supply current after compensation
axes[2].plot(t_ms, i_supply, 'b-', linewidth=1.5, label='Supply current (after compensation)')
axes[2].plot(t_ms, u/20, 'b--', linewidth=1, alpha=0.5, label='Voltage (scaled)')
axes[2].set_ylabel('Current (A)')
axes[2].set_xlabel('Time (ms)')
axes[2].set_title('Supply Current After Compensation (Should be in phase with voltage)')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Verify Compensation Results

In [ ]:
# Analyze AFTER compensation
gp_after = GeometricPower(u, i_supply, f1=f1, fs=fs)

print("\nCOMPARISON: BEFORE vs AFTER COMPENSATION")
print("=" * 60)
print(f"{'Quantity':<25} {'Before':<15} {'After':<15} {'Change':<15}")
print("-" * 60)

quantities = [
    ('Active Power P (W)', gp_before.P, gp_after.P),
    ('Reactive Power (var)', gp_before.M_Q_norm, gp_after.M_Q_norm),
    ('Distortion Power (va)', gp_before.M_D_norm, gp_after.M_D_norm),
    ('Apparent Power S (VA)', gp_before.S, gp_after.S),
    ('Power Factor', gp_before.PF, gp_after.PF),
    ('Supply Current (A)', rms(i), rms(i_supply)),
]

for name, before, after in quantities:
    if 'Factor' in name:
        change = f"+{(after-before)*100:.1f}%" if after > before else f"{(after-before)*100:.1f}%"
    else:
        pct_change = (after - before) / before * 100 if before != 0 else 0
        change = f"{pct_change:+.1f}%"
    print(f"{name:<25} {before:<15.2f} {after:<15.2f} {change:<15}")

print("=" * 60)

## 5. Partial Compensation Strategies

Full compensation may not always be practical or desirable. Let's explore partial compensation options.

In [ ]:
# Strategy 1: Compensate only fundamental reactive (like a capacitor bank)
def compensate_fundamental_reactive(u, i, f1, fs):
    """Compensate only fundamental frequency reactive power."""
    from scipy.fft import fft, ifft, fftfreq
    
    N = len(u)
    U_fft = fft(u)
    I_fft = fft(i)
    freqs = fftfreq(N, 1/fs)
    
    # Find fundamental
    idx1 = np.argmin(np.abs(freqs - f1))
    
    # Extract fundamental components
    U1 = U_fft[idx1]
    I1 = I_fft[idx1]
    
    # Calculate reactive current at fundamental
    phi = np.angle(U1) - np.angle(I1)
    I1_reactive = np.abs(I1) * np.sin(phi) * np.exp(1j * (np.angle(U1) - np.pi/2))
    
    # Create compensation current spectrum (only fundamental)
    I_comp_fft = np.zeros_like(I_fft)
    I_comp_fft[idx1] = -I1_reactive
    I_comp_fft[-idx1] = np.conj(-I1_reactive)  # Negative frequency
    
    return np.real(ifft(I_comp_fft))

# Strategy 2: Compensate only 5th harmonic
def compensate_harmonic(u, i, f1, fs, harmonic=5):
    """Compensate specific harmonic."""
    from scipy.fft import fft, ifft, fftfreq
    
    N = len(u)
    I_fft = fft(i)
    freqs = fftfreq(N, 1/fs)
    
    f_h = harmonic * f1
    idx_h = np.argmin(np.abs(freqs - f_h))
    
    # Create compensation current spectrum
    I_comp_fft = np.zeros_like(I_fft)
    I_comp_fft[idx_h] = -I_fft[idx_h]
    if idx_h > 0:
        I_comp_fft[-idx_h] = -I_fft[-idx_h]
    
    return np.real(ifft(I_comp_fft))

# Apply different strategies
i_comp_fund = compensate_fundamental_reactive(u, i, f1, fs)
i_comp_h5 = compensate_harmonic(u, i, f1, fs, harmonic=5)

# Calculate resulting supply currents
i_supply_fund = i + i_comp_fund
i_supply_h5 = i + i_comp_h5
i_supply_fund_h5 = i + i_comp_fund + i_comp_h5

# Analyze each
gp_fund = GeometricPower(u, i_supply_fund, f1=f1, fs=fs)
gp_h5 = GeometricPower(u, i_supply_h5, f1=f1, fs=fs)
gp_fund_h5 = GeometricPower(u, i_supply_fund_h5, f1=f1, fs=fs)

print("PARTIAL COMPENSATION STRATEGIES")
print("=" * 70)
print(f"\n{'Strategy':<30} {'PF':<10} {'Q (var)':<12} {'Comp. I (A)':<12}")
print("-" * 70)
print(f"{'No compensation':<30} {gp_before.PF:<10.4f} {gp_before.M_Q_norm:<12.2f} {0:<12.2f}")
print(f"{'Fund. reactive only':<30} {gp_fund.PF:<10.4f} {gp_fund.M_Q_norm:<12.2f} {rms(i_comp_fund):<12.2f}")
print(f"{'5th harmonic only':<30} {gp_h5.PF:<10.4f} {gp_h5.M_Q_norm:<12.2f} {rms(i_comp_h5):<12.2f}")
print(f"{'Fund. + 5th harmonic':<30} {gp_fund_h5.PF:<10.4f} {gp_fund_h5.M_Q_norm:<12.2f} {rms(i_comp_fund+i_comp_h5):<12.2f}")
print(f"{'Full compensation':<30} {gp_after.PF:<10.4f} {gp_after.M_Q_norm:<12.2f} {rms(i_comp_time):<12.2f}")
print("=" * 70)

## 6. Trade-offs Visualization

In [ ]:
# Plot compensation current requirements vs power factor improvement
strategies = [
    ('None', 0, gp_before.PF),
    ('Fund. Q', rms(i_comp_fund), gp_fund.PF),
    ('h=5', rms(i_comp_h5), gp_h5.PF),
    ('Fund.+h5', rms(i_comp_fund+i_comp_h5), gp_fund_h5.PF),
    ('Full', rms(i_comp_time), gp_after.PF),
]

fig, ax = plt.subplots(figsize=(10, 6))

names = [s[0] for s in strategies]
comp_currents = [s[1] for s in strategies]
power_factors = [s[2] for s in strategies]

ax.bar(names, power_factors, color='steelblue', alpha=0.7, label='Power Factor')

# Add compensation current as secondary axis
ax2 = ax.twinx()
ax2.plot(names, comp_currents, 'ro-', linewidth=2, markersize=10, label='Comp. Current')

ax.set_ylabel('Power Factor', color='steelblue')
ax2.set_ylabel('Compensation Current RMS (A)', color='red')
ax.set_xlabel('Compensation Strategy')
ax.set_title('Trade-off: Power Factor Improvement vs Compensation Current')

ax.set_ylim(0.7, 1.05)
ax.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='Unity PF')

# Combine legends
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='center right')

ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nKey insight: Full compensation achieves unity PF but requires")
print("the largest compensation current. Partial strategies may be more practical.")

## 7. Summary: Compensation Design Process

### GAPoT Compensation Workflow

1. **Measure** voltage and current waveforms
2. **Transform** to GA vectors using Fourier-to-GA
3. **Decompose** current: i = i_P + i_N
4. **Analyze** contributions: reactive (M_Q) vs distortion (M_D)
5. **Design** compensation based on:
   - Target power factor
   - Available compensation current capacity
   - Harmonic regulations (IEEE 519)
6. **Implement**: i_comp = -i_N (full) or partial

### Advantages of GA Approach

| Traditional | GAPoT |
|-------------|-------|
| P, Q, D separate | Unified M = ui |
| Reactive = Im{VI*} | Geometric projection |
| Distortion aggregated | Individual D_mn terms |
| Limited insight | Clear geometric picture |